In [1]:
import pandas as pd
import os
import re
import numpy as np

# EMICRON 22

### 1. Lectura información

In [2]:
data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Crudos"
data_res = r"D:\DANE - Contrato\2024\Operativo Barrido\Data"

In [3]:
emicron_22 = pd.read_sas(data_path + "\emicron_2022.sas7bdat", format = "sas7bdat")
# pd.DataFrame(emicron_22.columns).to_excel("Variables.xlsx")
# emicron_22[:5].transpose().to_excel("Muestra.xlsx")

In [4]:
emicron_22.shape

(84753, 495)

#### 1.1 Arreglo variables

In [5]:
# Construcción id
emicron_22['id'] = emicron_22['DIRECTORIO'].astype(int).astype(str) + emicron_22['SECUENCIA_P'].astype(int).astype(str) + emicron_22['ORDEN'].astype(int).astype(str)

C:\Users\carlo\AppData\Local\Temp\ipykernel_2848\104079840.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  emicron_22['id'] = emicron_22['DIRECTORIO'].astype(int).astype(str) + emicron_22['SECUENCIA_P'].astype(int).astype(str) + emicron_22['ORDEN'].astype(int).astype(str)


In [6]:
# pd.DataFrame(emicron_22[['VENTAS_MES_ANTERIOR','P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093',
#             'REMUNERACION_TOTAL',
#             'GASTOS_MES','P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K',
#             'COSTOS_MES_ANTERIOR','P3056_A','P3056_B','P3056_C','P3056_D',
#             'P3032_1','P3032_2','P3032_3',
#             'COD_DEPTO','area','GRUPOS4','P3053','P1633','P1055','P640','P639','P3034']].isna().sum())

pd.DataFrame(emicron_22[[#'VENTAS_MES_ANTERIOR','P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093',
           'REMUNERACION_TOTAL',
           'GASTOS_MES','P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K',
           'COSTOS_MES_ANTERIOR','P3056_A','P3056_B','P3056_C','P3056_D',
           'P3032_1','P3032_2','P3032_3',
           'COD_DEPTO','area','GRUPOS4','P3053','P1633','P1055','P640','P639','P3034','MPIO','GRUPOS12','CIIU_3','P2760']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84753 entries, 0 to 84752
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   REMUNERACION_TOTAL   84753 non-null  float64
 1   GASTOS_MES           84753 non-null  float64
 2   P3017_A              84753 non-null  float64
 3   P3017_B              84753 non-null  float64
 4   P3017_C              84753 non-null  float64
 5   P3017_D              84753 non-null  float64
 6   P3017_E              84753 non-null  float64
 7   P3017_F              84753 non-null  float64
 8   P3017_G              84753 non-null  float64
 9   P3017_H              84753 non-null  float64
 10  P3017_K              84753 non-null  float64
 11  COSTOS_MES_ANTERIOR  84753 non-null  float64
 12  P3056_A              21803 non-null  float64
 13  P3056_B              44510 non-null  float64
 14  P3056_C              7653 non-null   float64
 15  P3056_D              10767 non-null 

""


In [7]:
emicron_22['GRUPOS12'] = emicron_22['GRUPOS12'].astype('int')
emicron_22['GRUPOS4'] = emicron_22['GRUPOS4'].astype('int')
emicron_22['MPIO'] = emicron_22['MPIO'].astype('int')
emicron_22['COD_DEPTO'] = emicron_22['COD_DEPTO'].astype('int')

### 2. Filtrado de la información
Se procederá a filtrar la base de datos con base en unos elementos particulares:  
1. Actividades económicas que cubre EMICRON y que no serán tenidas en cuenta en CENU.  
2. Ubicación del micronegocio. Ya que se entrenará un modelo para puestos fijos y micronegocios en vivienda (sólo los visibles al público), y otro para puestos móviles.   
3. Otras variables como el número de establecimientos, que permitan mantener una información homogénea.

#### 2.1 Actividades económicas que cubrirá el CENU 
Emicron incluye preguntas a los sectores de agricultura, ganadería, caza, silvicultura y pesca; además, de minería. Debido a que estos sectores no harán parte del Universo del Censo Económico, se descartarán de la base de datos:

In [8]:
emicron_22 = emicron_22[~(emicron_22['GRUPOS12'].isin([1,2,13]))]
emicron_22.shape

(73966, 496)

In [9]:
emicron_22['GRUPOS12'].value_counts()

5     24123
6     11513
3      9045
12     8779
7      7494
4      6156
9      4627
11      936
10      770
8       523
Name: GRUPOS12, dtype: int64

#### 2.2 Ubicación del micronegocio
Para el entrenamiento de los modelos se utilizará la ubicación de micronegocios en 1: Vivienda, 2: Local (puestos fijos), 4: Ambulante, por lo tanto sólo estas categorías se mantendrán. De igual manera, el CENU, sólo recolectará información de actividades económicas visibles, por lo tanto se filtrará por este tipo de micronegocios.

In [10]:
print(emicron_22['P3053'].value_counts())

# Seleccionar sólo viviendas, establecimientos y ambulantes
emicron_22 = emicron_22[emicron_22['P3053'].isin([1,2,4])]

print(emicron_22['P3053'].value_counts())

1.0    24206
3.0    15006
5.0    11263
2.0    10739
4.0     9184
6.0     2812
8.0      390
7.0      366
Name: P3053, dtype: int64
1.0    24206
2.0    10739
4.0     9184
Name: P3053, dtype: int64


In [11]:
# Visibilidad del micronegocio (sólo para viviendas)
emicron_22.drop(emicron_22[(emicron_22['P3053'] == 1) & (emicron_22['P469'] == 2)].index, inplace = True)

# [(emicron_22['P469'] == 1)]
print(emicron_22['P3053'].value_counts())

1.0    13395
2.0    10739
4.0     9184
Name: P3053, dtype: int64


In [12]:
emicron_22.shape

(33318, 496)

In [13]:
# Reemplazar los códigos en función de la codificación que se recibirá en el CENU
# 1: Establecimiento fijo
# 3: Vivienda con actividad económica visible
def rule(i):
    if i == 1:
        return 3
    elif i == 2:
        return 1
    return i

emicron_22['P3053'] = list(map(rule,emicron_22['P3053']))
print(emicron_22['P3053'].value_counts())

3.0    13395
1.0    10739
4.0     9184
Name: P3053, dtype: int64


#### 2.3 Ventas del mes anterior
Eliminar las filas cuyas ventas del año pasado son missing y cero

In [14]:
# Valores nulos ventas
print("Valores nulos ventas: ", emicron_22["VENTAS_MES_ANTERIOR"].isna().sum())

# Valores cero ventas
print("Ventas en cero: ", (emicron_22["VENTAS_MES_ANTERIOR"]==0).sum())

# Eliminar los registros con venta cero
#emicron_22 = emicron_22[~(emicron_22["VENTAS_MES_ANTERIOR"] == 0)]

emicron_22.shape

Valores nulos ventas:  0
Ventas en cero:  817


(33318, 496)

#### 2.4 Número de establecimientos
Eliminar las filas cuyo número de estabecimientos es diferente a 1, ya que la información que recolecta el CENU es por establecimiento y la recolectada por EMICRON es a nivel de negocio, pero a veces se presenta información también solo por establecimiento cuando el negocio tiene más de un establecimiento. Para evitar eso, sólo utilizar los micronegocios que reportan un sólo establecimiento comercial, que representan el 99.36 % de la muestra (casi todos).

In [15]:
emicron_22['P3054'] = pd.to_numeric(emicron_22['P3054'], errors='coerce')
print(emicron_22['P3054'].value_counts(),"\n_____________\n")
emicron_22 = emicron_22[(emicron_22['P3054'] == 1) | ((emicron_22['P3054'].isna()) & (emicron_22['P3053']==4))]
emicron_22.shape

1.0     27363
2.0        99
3.0        13
99.0       12
4.0         2
9.0         1
Name: P3054, dtype: int64 
_____________



(33191, 496)

In [16]:
pd.crosstab(emicron_22['P3054'].isna(), emicron_22['P3053'])

P3053,1.0,3.0,4.0
P3054,,,
False,10655,13371,3337
True,0,0,5828


In [17]:
pd.crosstab(emicron_22['P469'], emicron_22['P3053'])

P3053,1.0,3.0,4.0
P469,,,
1.0,9972,13371,3129
2.0,683,0,208


### 3. Creación variables principales
1. Se creará una agrupación para la variable del número total de empleados  
2. Se agregarán los costos y los gastos en una sola variable (CYG)  
3. AREA se recodificará para que muestre si la observación pertenece a una ciudad capital del departamento  
4. Contabilidad

#### 3.1 Personal Ocupado

In [18]:
emicron_22['POTTOT'] = emicron_22[['P3032_1','P3032_2','P3032_3']].sum(axis=1) + 1
#emicron_22['POT_rem'] = emicron_22[['P3032_1']].sum(axis=1) + 1
#emicron_22['POT_norem'] = emicron_22[['P3032_2','P3032_3']].sum(axis=1)

# emicron_22[['POTTOT','POT_rem','POT_norem']].describe()
# emicron_22[['POTTOT','POT_rem','POT_norem','REMUNERACION_TOTAL']].corr()

#print("Coincidentes entre POTTOT y desagregación propuesta:\n",(emicron_22['POTTOT'] == emicron_22[['POT_rem','POT_norem']].sum(axis=1)).value_counts())

In [19]:
emicron_22['POTTOT'].describe()

count    33191.000000
mean         1.386611
std          0.867077
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          9.000000
Name: POTTOT, dtype: float64

#### 3.2 Ventas al mes

In [20]:
# Comparación suma individual y la agregada
print("Número de filas coincidentes entre la variable agregada y la suma de las individuales:\n",(emicron_22['VENTAS_MES_ANTERIOR'] == emicron_22[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']].sum(axis=1)).value_counts())
#emicron_22[['VENTAS_MES_ANTERIOR','P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']][~(emicron_22['VENTAS_MES_ANTERIOR'] == emicron_22[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']].sum(axis=1))]

Número de filas coincidentes entre la variable agregada y la suma de las individuales:
 True    33191
dtype: int64


#### 3.3 Costos y gastos al mes

In [21]:
# Comparación suma individual y la agregada
print("Número de filas coincidentes entre la variable agregada y la suma de las individuales:\n",(emicron_22['COSTOS_MES_ANTERIOR'] == emicron_22[['P3056_A','P3056_B','P3056_C','P3056_D']].sum(axis=1)).value_counts())

Número de filas coincidentes entre la variable agregada y la suma de las individuales:
 True    33191
dtype: int64


In [22]:
# Comparación suma individual y la agregada
print("Número de filas coincidentes entre la variable agregada y la suma de las individuales:\n",(emicron_22['GASTOS_MES'] == emicron_22[['P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K']].sum(axis=1)).value_counts())

Número de filas coincidentes entre la variable agregada y la suma de las individuales:
 True    33191
dtype: int64


In [23]:
# Agregación de la variable
emicron_22['CYG'] = emicron_22[['COSTOS_MES_ANTERIOR','GASTOS_MES']].sum(axis=1)

#### 3.4 Variable de Ciudad Capital y A.M
Recodificar la variable de Área en función de si es una ciudad principal o un área metropolitana dentro del departamento

In [24]:
def rule_city(i):
    if pd.isna(i) == True:
        return 0
    else:
        return 1
    return i

emicron_22['CPAM'] = list(map(rule_city,emicron_22['area']))

In [25]:
pd.crosstab(emicron_22['COD_DEPTO'],emicron_22['area'].isna())

area,False,True
COD_DEPTO,,
5,1487,419
8,1343,434
11,663,11
13,1392,350
15,659,260
17,1018,269
18,1249,261
19,723,263
20,843,246


#### 3.5 Variables de formalidad

In [26]:
## Formalidad del micronegocio
# Tiene RUT
emicron_22['P1633'] = [0 if num == 2 else num for num in emicron_22['P1633']]
# Está inscrito en CC
emicron_22['P1055'] = [0 if num == 2 else num for num in emicron_22['P1055']]

#### 3.6 Manera de llevar la contabilidad

In [27]:
## Manera de llevar contabilidad
# Se unifica 4: informes financieros en 1: Balance General P&G
print("Frecuencia forma de llevar contabilidad emicron:\n",emicron_22['P640'].value_counts(),"__________\n")

emicron_22['P640'] = [1 if num == 4 else num for num in emicron_22['P640']]
print("Frecuencia luego de la homologación:\n",emicron_22['P640'].value_counts(),"________\n")

# Se cambia el 5 por 4, para homogenizar con los códigos de CENU
emicron_22['P640'] = [4 if num == 5 else num for num in emicron_22['P640']]
print("Frecuencia luego del cambio de códificación:\n",emicron_22['P640'].value_counts(),"________\n")

Frecuencia forma de llevar contabilidad emicron:
 5.0    18080
3.0    11317
2.0     2626
1.0     1034
4.0      134
Name: P640, dtype: int64 __________

Frecuencia luego de la homologación:
 5.0    18080
3.0    11317
2.0     2626
1.0     1168
Name: P640, dtype: int64 ________

Frecuencia luego del cambio de códificación:
 4.0    18080
3.0    11317
2.0     2626
1.0     1168
Name: P640, dtype: int64 ________



#### 3.7 Antigüedad

In [28]:
emicron_22['ANIOS_OPERACION'] = emicron_22['P3034'] / 12
#emicron_22['ANIOS_OPERACION'].describe()
emicron_22['ANIOS_OPERACION'] = round(emicron_22['ANIOS_OPERACION'])
emicron_22['ANIOS_OPERACION'].describe()

count    33191.000000
mean        10.077973
std         10.228451
min          0.000000
25%          2.000000
50%          6.000000
75%         15.000000
max         83.000000
Name: ANIOS_OPERACION, dtype: float64

#### 3.8 Sexo

In [29]:
emicron_22['P35'].value_counts()
emicron_22['P35'] = [0 if num == 2 else num for num in emicron_22['P35']]

## 4. Renombrar variables

In [30]:
nombre_estandar = {'P3053': 'UBICA',
                   'P1633': 'IDRUT',
                   'P640': 'REG_CONT',
                   'P1055':'CAMCOMER',
                   'P639' : 'ANOS_ESPACIO',
                   'P35' : 'HOMBRE'
               }

emicron_22.rename(columns=nombre_estandar, inplace=True)

In [31]:
emicron_22['OLA'] = 2022
emicron_22.to_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_22_vf.csv", index= False)

In [33]:
# emicron_22 = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_22_vf.csv", header=0)    
# (emicron_22['REMUNERACION_TOTAL'] == emicron_22[['SUELDOS','PRESTACIONES']].sum(axis = 1)).value_counts()
# emicron_22[['id','fex','fex_micro_c','FEX_C','FEX_P','F_EXP']]

In [34]:
# data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data"
# emicron_22_pub = pd.read_csv(os.path.join(data_path,"EMICRON_22.csv"),header = 0)
# emicron_22 = pd.merge(emicron_22, emicron_22_pub[['id','F_EXP']], on='id')

In [32]:
var_estab = ['id','VENTAS_MES_ANTERIOR','CYG','POTTOT','REMUNERACION_TOTAL','COD_DEPTO','CPAM','GRUPOS4','UBICA','IDRUT','CAMCOMER','REG_CONT','ANIOS_OPERACION'] 

var_mov = ['id','VENTAS_MES_ANTERIOR','POTTOT','REMUNERACION_TOTAL','CYG','COD_DEPTO','CPAM','GRUPOS4','ANOS_ESPACIO','HOMBRE']

In [41]:
# ¿Existe alguna fila con valores nulos == 99?
((emicron_22[var_estab]==99).any(axis=1)).value_counts()

False    33191
dtype: int64

# EMICRON 23

### 1. Lectura información

In [6]:
# data_path = r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Crudos"

In [3]:
emicron_23 = pd.read_sas(os.path.join(data_path,"emicron_2023 (2).sas7bdat"), format = "sas7bdat")
# pd.DataFrame(emicron_23.columns).to_excel("Variables.xlsx")
# emicron_23[:5].transpose().to_excel("Muestra.xlsx")

In [4]:
emicron_23.shape

(864144, 518)

In [5]:
#emicron_23_ = emicron_23.copy()
emicron_23 = emicron_23[emicron_23['micro']==1]

In [6]:
emicron_23.shape

(81018, 518)

In [7]:
emicron_23.to_csv(data_res + "\EMICRON_Sergio.csv", index = False, sep = ";")

#### 1.1 Arreglo variables

In [11]:
# Construcción id
emicron_23['id'] = emicron_23['DIRECTORIO'].astype(int).astype(str) + emicron_23['SECUENCIA_P'].astype(int).astype(str) + emicron_23['orden'].astype(int).astype(str)

C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\1834767636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emicron_23['id'] = emicron_23['DIRECTORIO'].astype(int).astype(str) + emicron_23['SECUENCIA_P'].astype(int).astype(str) + emicron_23['orden'].astype(int).astype(str)


In [14]:
# pd.DataFrame(emicron_23[['VENTAS_MES_ANTERIOR','P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093',
#             'REMUNERACION_TOTAL',
#             'GASTOS_MES','P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K',
#             'COSTOS_MES_ANTERIOR','P3056_A','P3056_B','P3056_C','P3056_D',
#             'P3032_1','P3032_2','P3032_3',
#             'COD_DEPTO','area','GRUPOS4','P3053','P1633','P1055','P640','P639','P3034']].isna().sum())

pd.DataFrame(emicron_23[['VENTAS_MES_ANTERIOR',#'P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093',
           'REMUNERACION_TOTAL',
           'GASTOS_MES',#'P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K',
           'COSTOS_MES_ANTERIOR',#'P3056_A','P3056_B','P3056_C','P3056_D',
           'P3032_1','P3032_2','P3032_3',
           'COD_DEPTO','area','GRUPOS4','P3053','P3054','P1633','P1055','P640','P639','P3034','MPIO','GRUPOS12','CIIU_3','P2760']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81018 entries, 0 to 81017
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   VENTAS_MES_ANTERIOR  0 non-null      float64
 1   REMUNERACION_TOTAL   0 non-null      float64
 2   GASTOS_MES           0 non-null      float64
 3   COSTOS_MES_ANTERIOR  0 non-null      float64
 4   P3032_1              13498 non-null  float64
 5   P3032_2              13496 non-null  float64
 6   P3032_3              13496 non-null  float64
 7   COD_DEPTO            81018 non-null  object 
 8   area                 55388 non-null  float64
 9   GRUPOS4              81018 non-null  object 
 10  P3053                81018 non-null  float64
 11  P3054                55786 non-null  float64
 12  P1633                81018 non-null  float64
 13  P1055                81018 non-null  float64
 14  P640                 81018 non-null  float64
 15  P639                 81017 non-null 

""


In [12]:
emicron_23['GRUPOS12'] = emicron_23['GRUPOS12'].astype('int')
emicron_23['GRUPOS4'] = emicron_23['GRUPOS4'].astype('int')
emicron_23['MPIO'] = emicron_23['MPIO'].astype('int')
emicron_23['COD_DEPTO'] = emicron_23['COD_DEPTO'].astype('int')

C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\1245985555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emicron_23['GRUPOS12'] = emicron_23['GRUPOS12'].astype('int')
C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\1245985555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emicron_23['GRUPOS4'] = emicron_23['GRUPOS4'].astype('int')
C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\1245985555.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

### 2. Filtrado de la información
Se procederá a filtrar la base de datos con base en unos elementos particulares:  
1. Actividades económicas que cubre EMICRON y que no serán tenidas en cuenta en CENU.  
2. Ubicación del micronegocio. Ya que se entrenará un modelo para puestos fijos y micronegocios en vivienda (sólo los visibles al público), y otro para puestos móviles.   
3. Otras variables como el número de establecimientos, que permitan mantener una información homogénea.

#### 2.1 Actividades económicas que cubrirá el CENU 
Emicron incluye preguntas a los sectores de agricultura, ganadería, caza, silvicultura y pesca; además, de minería. Debido a que estos sectores no harán parte del Universo del Censo Económico, se descartarán de la base de datos:

In [13]:
emicron_23 = emicron_23[~(emicron_23['GRUPOS12'].isin([1,2,13]))]
emicron_23.shape

(70597, 519)

In [14]:
emicron_23['GRUPOS12'].value_counts()

5     21828
6     11715
12     8898
3      8372
7      7430
4      5555
9      4598
11      903
10      826
8       472
Name: GRUPOS12, dtype: int64

#### 2.2 Ubicación del micronegocio
Para el entrenamiento de los modelos se utilizará la ubicación de micronegocios en 1: Vivienda, 2: Local (puestos fijos), 4: Ambulante, por lo tanto sólo estas categorías se mantendrán. De igual manera, el CENU, sólo recolectará información de actividades económicas visibles, por lo tanto se filtrará por este tipo de micronegocios.

In [15]:
print(emicron_23['P3053'].value_counts())

# Seleccionar sólo viviendas, establecimientos y ambulantes
emicron_23 = emicron_23[emicron_23['P3053'].isin([1,2,4])]

print(emicron_23['P3053'].value_counts())

1.0    23469
3.0    13563
5.0    11383
2.0    10043
4.0     8662
6.0     2691
8.0      456
7.0      330
Name: P3053, dtype: int64
1.0    23469
2.0    10043
4.0     8662
Name: P3053, dtype: int64


In [16]:
emicron_23['GRUPOS12'].value_counts()

5     17483
3      7448
7      7005
12     5498
9      2831
11      631
10      491
8       375
6       225
4       187
Name: GRUPOS12, dtype: int64

In [17]:
# Visibilidad del micronegocio (sólo eliminar viviendas no visibles al público)
emicron_23.drop(emicron_23[(emicron_23['P3053'] == 1) & (emicron_23['P469'] == 2)].index, inplace = True)

# [(emicron_23['P469'] == 1)]
print(emicron_23['P3053'].value_counts())

1.0    12685
2.0    10043
4.0     8662
Name: P3053, dtype: int64


In [18]:
emicron_23.shape

(31390, 519)

In [19]:
# Reemplazar los códigos en función de la codificación que se recibirá en el CENU
# 1: Establecimiento fijo
# 3: Vivienda con actividad económica visible
def rule(i):
    if i == 1:
        return 3
    elif i == 2:
        return 1
    return i

emicron_23['P3053'] = list(map(rule,emicron_23['P3053']))
print(emicron_23['P3053'].value_counts())

3.0    12685
1.0    10043
4.0     8662
Name: P3053, dtype: int64


#### 2.3 Ventas del mes anterior
Eliminar las filas cuyas ventas del año pasado son missing y cero

In [20]:
# Valores nulos ventas
print("Valores nulos ventas: ", emicron_23["VENTAS_MES_ANTERIOR"].isna().sum())

# Valores cero ventas
print("Ventas en cero: ", (emicron_23["VENTAS_MES_ANTERIOR"]==0).sum())

# Eliminar los registros con venta cero
#emicron_23 = emicron_23[~(emicron_23["VENTAS_MES_ANTERIOR"] == 0)]

emicron_23.shape

Valores nulos ventas:  0
Ventas en cero:  618


(31390, 519)

#### 2.4 Número de establecimientos
Eliminar las filas cuyo número de estabecimientos es diferente a 1, ya que la información que recolecta el CENU es por establecimiento y la recolectada por EMICRON es a nivel de negocio, pero a veces se presenta información también solo por establecimiento cuando el negocio tiene más de un establecimiento. Para evitar eso, sólo utilizar los micronegocios que reportan un sólo establecimiento comercial, que representan el 99.36 % de la muestra (casi todos).

In [21]:
pd.crosstab(emicron_23['P3054'].isna(), emicron_23['P3053'])

P3053,1.0,3.0,4.0
P3054,,,
False,10043,12685,3450
True,0,0,5212


In [22]:
emicron_23['P3054'] = pd.to_numeric(emicron_23['P3054'], errors='coerce')
print(emicron_23['P3054'].value_counts(),"\n_____________\n")
emicron_23 = emicron_23[(emicron_23['P3054'] == 1) | ((emicron_23['P3054'].isna()) & (emicron_23['P3053']==4))]
emicron_23.shape

1.0     26078
2.0        77
3.0        19
11.0        2
6.0         1
5.0         1
Name: P3054, dtype: int64 
_____________



(31290, 519)

In [23]:
pd.crosstab(emicron_23['P3054'].isna(), emicron_23['P3053'])

P3053,1.0,3.0,4.0
P3054,,,
False,9974,12669,3435
True,0,0,5212


In [24]:
pd.crosstab(emicron_23['P469'], emicron_23['P3053'])

P3053,1.0,3.0,4.0
P469,,,
1.0,9374,12669,3195
2.0,600,0,240


### 3. Creación variables principales
1. Se creará una agrupación para la variable del número total de empleados  
2. Se agregarán los costos y los gastos en una sola variable (CYG)  
3. AREA se recodificará para que muestre si la observación pertenece a una ciudad capital del departamento  
4. Contabilidad

#### 3.1 Personal Ocupado

In [25]:
emicron_23['POTTOT'] = emicron_23[['P3032_1','P3032_2','P3032_3']].sum(axis=1) + 1
#emicron_23['POT_rem'] = emicron_23[['P3032_1']].sum(axis=1) + 1
#emicron_23['POT_norem'] = emicron_23[['P3032_2','P3032_3']].sum(axis=1)

# emicron_23[['POTTOT','POT_rem','POT_norem']].describe()
# emicron_23[['POTTOT','POT_rem','POT_norem','REMUNERACION_TOTAL']].corr()

#print("Coincidentes entre POTTOT y desagregación propuesta:\n",(emicron_23['POTTOT'] == emicron_23[['POT_rem','POT_norem']].sum(axis=1)).value_counts())

In [26]:
emicron_23[emicron_23['P3053']!=4]['POTTOT'].describe()

count    22643.000000
mean         1.496666
std          1.011429
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          9.000000
Name: POTTOT, dtype: float64

#### 3.2 Ventas al mes

In [27]:
# Comparación suma individual y la agregada
print("Número de filas coincidentes entre la variable agregada y la suma de las individuales:\n",(emicron_23['VENTAS_MES_ANTERIOR'] == emicron_23[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']].sum(axis=1)).value_counts())
#emicron_23[['VENTAS_MES_ANTERIOR','P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']][~(emicron_23['VENTAS_MES_ANTERIOR'] == emicron_23[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']].sum(axis=1))]

Número de filas coincidentes entre la variable agregada y la suma de las individuales:
 True    31290
dtype: int64


In [28]:
# ¿Existen filas que no cuenten con información?
print("Información datos\n:",emicron_23.shape,"\n")
(emicron_23[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']].isna().all(axis = 1)).value_counts()
# Parece que no existe filas con NA's

Información datos
: (31290, 520) 



False    31290
dtype: int64

In [34]:
((emicron_23[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']]==99).any(axis = 1)).value_counts()

False    31290
dtype: int64

In [55]:
#emicron_23['VENTAS_MES_ANTERIOR'] = emicron_23[['P3057','P3058','P3059','P3060','P3061','P3062','P4002','P3063','P3064','P3065','P3066','P3067','P3092','P3093']].sum(axis=1)

#### 3.3 Costos y gastos al mes

In [35]:
# Comparación suma individual y la agregada
print("Número de filas coincidentes entre la variable agregada y la suma de las individuales:\n",(emicron_23['COSTOS_MES_ANTERIOR'] == emicron_23[['P3056_A','P3056_B','P3056_C','P3056_D']].sum(axis=1)).value_counts())

Número de filas coincidentes entre la variable agregada y la suma de las individuales:
 True    31290
dtype: int64


In [36]:
# ¿Existen filas que no cuenten con información?
print("Información datos\n:",emicron_23.shape,"\n")
(emicron_23[['P3056_A','P3056_B','P3056_C','P3056_D']].isna().all(axis = 1)).value_counts()

# Parece que no existe filas con NA's

Información datos
: (31290, 520) 



False    31290
dtype: int64

In [37]:
((emicron_23[['P3056_A','P3056_B','P3056_C','P3056_D']]==99).all(axis = 1)).value_counts()

False    31290
dtype: int64

In [58]:
#emicron_23['COSTOS_MES_ANTERIOR'] = emicron_23[['P3056_A','P3056_B','P3056_C','P3056_D']].sum(axis=1)

In [38]:
# Comparación suma individual y la agregada
print("Número de filas coincidentes entre la variable agregada y la suma de las individuales:\n",(emicron_23['GASTOS_MES'] == emicron_23[['P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K']].sum(axis=1)).value_counts())

Número de filas coincidentes entre la variable agregada y la suma de las individuales:
 True    31290
dtype: int64


In [39]:
# ¿Existen filas que no cuenten con información?
print("Información datos\n:",emicron_23.shape,"\n")
(emicron_23[['P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K']].isna().all(axis = 1)).value_counts()

# Parece que no existe filas con NA's

Información datos
: (31290, 520) 



False    31290
dtype: int64

In [40]:
((emicron_23[['P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K']]==99).all(axis = 1)).value_counts()

False    31290
dtype: int64

In [61]:
#emicron_23['GASTOS_MES'] = emicron_23[['P3017_A','P3017_B','P3017_C','P3017_D','P3017_E','P3017_F','P3017_G','P3017_H','P3017_K']].sum(axis=1)

In [41]:
# Agregación de la variable
emicron_23['CYG'] = emicron_23[['COSTOS_MES_ANTERIOR','GASTOS_MES']].sum(axis=1)

#### 3.4 Remuneración Total

In [44]:
print("Valores nulos: remuneración",emicron_23['REMUNERACION_TOTAL'].isna().sum(),"\n_________________")
print("Valores nulos: prestaciones",emicron_23['PRESTACIONES'].isna().sum(),"\n_________________")
print("Valores nulos: sueldos",emicron_23['SUELDOS'].isna().sum(),"\n_________________")

Valores nulos: remuneración 0 
_________________
Valores nulos: prestaciones 0 
_________________
Valores nulos: sueldos 23670 
_________________


In [46]:
(emicron_23['REMUNERACION_TOTAL']==99).sum()

0

#### 3.5 Variable de Ciudad Capital y A.M
Recodificar la variable de Área en función de si es una ciudad principal o un área metropolitana dentro del departamento

In [47]:
def rule_city(i):
    if pd.isna(i) == True:
        return 0
    else:
        return 1
    return i

emicron_23['CPAM'] = list(map(rule_city,emicron_23['area']))

In [48]:
pd.crosstab(emicron_23['COD_DEPTO'],emicron_23['CPAM'])

CPAM,0,1
COD_DEPTO,,
5,592,1482
8,329,1404
11,19,707
13,292,1418
15,216,708
17,229,904
18,293,1069
19,288,943
20,242,788


In [49]:
pd.crosstab(emicron_23['COD_DEPTO'],emicron_23['area'].isna())

area,False,True
COD_DEPTO,,
5,1482,592
8,1404,329
11,707,19
13,1418,292
15,708,216
17,904,229
18,1069,293
19,943,288
20,788,242


#### 3.6 Variables de formalidad

In [50]:
## Formalidad del micronegocio
# Tiene RUT
emicron_23['P1633'] = [0 if num == 2 else num for num in emicron_23['P1633']]
# Está inscrito en CC
emicron_23['P1055'] = [0 if num == 2 else num for num in emicron_23['P1055']]

#### 3.7 Manera de llevar la contabilidad

In [51]:
emicron_23['P640'].value_counts()

5.0    17106
3.0    11002
2.0     2151
1.0      861
4.0      170
Name: P640, dtype: int64

In [52]:
## Manera de llevar contabilidad
# Se unifica 4: informes financieros en 1: Balance General P&G
print("Frecuencia forma de llevar contabilidad emicron:\n",emicron_23['P640'].value_counts(),"__________\n")

emicron_23['P640'] = [1 if num == 4 else num for num in emicron_23['P640']]
print("Frecuencia luego de la homologación:\n",emicron_23['P640'].value_counts(),"________\n")

# Se cambia el 5 por 4, para homogenizar con los códigos de CENU
emicron_23['P640'] = [4 if num == 5 else num for num in emicron_23['P640']]
print("Frecuencia luego del cambio de códificación:\n",emicron_23['P640'].value_counts(),"________\n")

Frecuencia forma de llevar contabilidad emicron:
 5.0    17106
3.0    11002
2.0     2151
1.0      861
4.0      170
Name: P640, dtype: int64 __________

Frecuencia luego de la homologación:
 5.0    17106
3.0    11002
2.0     2151
1.0     1031
Name: P640, dtype: int64 ________

Frecuencia luego del cambio de códificación:
 4.0    17106
3.0    11002
2.0     2151
1.0     1031
Name: P640, dtype: int64 ________



#### 3.8 Antigüedad

In [53]:
emicron_23['ANIOS_OPERACION'] = emicron_23['P3034'] / 12
#emicron_23['ANIOS_OPERACION'].describe()
emicron_23['ANIOS_OPERACION'] = round(emicron_23['ANIOS_OPERACION'])
emicron_23['ANIOS_OPERACION'].describe()

count    31290.000000
mean        10.558134
std         10.498007
min          0.000000
25%          3.000000
50%          7.000000
75%         15.000000
max         83.000000
Name: ANIOS_OPERACION, dtype: float64

#### 3.9 Sexo

In [54]:
emicron_23['P35'].value_counts()
emicron_23['P35'] = [0 if num == 2 else num for num in emicron_23['P35']]

## 4. Renombrar variables

In [55]:
nombre_estandar = {'P3053': 'UBICA',
                   'P1633': 'IDRUT',
                   'P640': 'REG_CONT',
                   'P1055':'CAMCOMER',
                   'P639' : 'ANOS_ESPACIO',
                   'P35' : 'HOMBRE'
               }

emicron_23.rename(columns=nombre_estandar, inplace=True)

In [56]:
emicron_23['OLA'] = 2023
emicron_23.to_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\emicron_23_vf.csv", index= False)

In [57]:
#emicron_22 = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\emicron_22_vf.csv")

C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\432892577.py:1: DtypeWarning: Columns (59,60,102,104,106,132,133,134,135,136,137,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  emicron_22 = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\emicron_22_vf.csv")


# Concatenado

In [60]:
emicron = pd.concat([emicron_22, emicron_23])

## Creación de variables
1. Valor agregado y coeficiente técnico de costos sobre ventas  
2. Variable dicótoma para la presencia de palabras clave en los comentarios del encuestador sobre la encuesta

### Valor agregado y coeficiente técnico

In [61]:
emicron['VA'] = emicron['VENTAS_MES_ANTERIOR'] - emicron['CYG']
emicron['C_V'] = emicron['CYG'] / (emicron['VENTAS_MES_ANTERIOR'] + 1)

C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\134861341.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  emicron['VA'] = emicron['VENTAS_MES_ANTERIOR'] - emicron['CYG']
C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\134861341.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  emicron['C_V'] = emicron['CYG'] / (emicron['VENTAS_MES_ANTERIOR'] + 1)


### Análisis variable de comentarios

In [62]:
import os
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
#nltk.download('punkt')
import spacy
import re

In [125]:
# Eliminar el byte estricto en la columna de comentarios y crear una nueva donde se realizará la limpieza
df_obs = emicron.copy()

df_obs['observaciones'] = df_obs['P2760'].apply(lambda x: str(x).strip('b').strip('"'))

# Reemplazar caracter especial de ñ
df_obs['observaciones'] = df_obs['observaciones'].apply(lambda x: x.replace("\\xd1","ñ"))

# Reemplazar 'nan' for NA
df_obs['observaciones'] = df_obs['observaciones'].replace('nan',np.NaN)

# Eliminar del data frame las filas sin comentarios
df_obs['observaciones'] = df_obs['observaciones'][~df_obs['observaciones'].isna()]

#Remover Puntuación
def remover_puntuacion(texto):
    return re.sub(r'[^\w\s]', '', texto)

# Remover números
def remover_numeros(texto):
    return re.sub(r'\d+', '', texto)


df_obs['observaciones'] = df_obs['observaciones'].apply(remover_puntuacion)
df_obs['observaciones'] = df_obs['observaciones'].apply(remover_numeros)

In [126]:
nlp = spacy.load("es_core_news_sm")

C:\Users\carlo\AppData\Roaming\Python\Python310\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [127]:
palabras = ['venta','ingreso','costo','gasto','insumos','ganancia','personal','error','corregir']

def detected_words(text):
    # Cargar el conjunto de stopwords
    stop_words = set(stopwords.words('spanish'))
    # Tokenizar las oraciones
    tokens = word_tokenize(text)
    # Eliminar stopwords
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    # Lematización
    lemmatized_tokens = [token.lemma_ for token in nlp(" ".join(filtered_tokens))]

    a = [i for i in palabras if i in lemmatized_tokens]

    b = any(i in lemmatized_tokens for i in palabras)
    
    return a,b

In [128]:
df_obs[['k_words','mencion']] = df_obs['observaciones'].apply(detected_words).apply(pd.Series)

In [150]:
emicron.shape

(64481, 530)

In [149]:
emicron = pd.merge(emicron, df_obs[['id','k_words','mencion']], on = 'id', how = 'left')

In [63]:
emicron_antes = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_VF.csv")

C:\Users\carlo\AppData\Local\Temp\ipykernel_38116\2427750931.py:1: DtypeWarning: Columns (59,60,102,104,106,119,132,133,134,135,136,137,140,141,143,157,363,377,388,393,401,420,485,502,503,506,524,525) have mixed types. Specify dtype option on import or set low_memory=False.
  emicron_antes = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_VF.csv")


In [64]:
emicron.shape

(64481, 528)

In [65]:
emicron = pd.merge(emicron, emicron_antes[['id','k_words','mencion']], on='id', how='left')

In [67]:
emicron.to_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_VF.csv", index= False)

### Revisión atípicos

In [9]:
emicron = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_VF.csv")
atpcs_estab = pd.read_excel(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Resultados etapa 2\cluster_atipico_establecimientos_con observaciones.xlsx")
atpcs_calle = pd.read_excel(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Resultados etapa 2\cluster_atipico_moviles_con observaciones.xlsx")

C:\Users\carlo\AppData\Local\Temp\ipykernel_20988\2829988077.py:1: DtypeWarning: Columns (59,60,102,104,106,119,132,133,134,135,136,137,140,141,143,157,363,377,388,393,401,420,485,502,503,506,524,525) have mixed types. Specify dtype option on import or set low_memory=False.
  emicron = pd.read_csv(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\EMICRON_VF.csv")


In [23]:
atpcs_calle.shape

(4809, 14)

In [28]:
emicron['ATPC'] = emicron['id'].isin(atpcs_estab['id'])
emicron.loc[emicron['id'].isin(atpcs_calle['id']),"ATPC"] = emicron.loc[emicron['id'].isin(atpcs_calle['id']),"ATPC"].replace(False,True)

In [158]:
emicron.to_csv(data_res + "\EMICRON_ATPCS.csv"), index= False)

KeyboardInterrupt: 

### Revisión muestra atípicos

In [33]:
muestra_estab = pd.read_excel(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Resultados etapa 2\muestra_seleccionada_establecimientos.xlsx")
muestra_calle = pd.read_excel(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Resultados etapa 2\muestra_seleccionada_moviles.xlsx")

In [38]:
muestra_estab['VALOR AGREGADO'] = muestra_estab['VENTAS_MES_ANTERIOR'] - muestra_estab['CYG']
muestra_estab['COSTOS_VENTAS'] = muestra_estab['CYG'] / (muestra_estab['VENTAS_MES_ANTERIOR']+1)

muestra_calle['VALOR AGREGADO'] = muestra_calle['VENTAS_MES_ANTERIOR'] - muestra_calle['CYG']
muestra_calle['COSTOS_VENTAS'] = muestra_calle['CYG'] / (muestra_calle['VENTAS_MES_ANTERIOR']+1)

In [39]:
print(muestra_estab.shape)

print(muestra_calle.shape)

(112, 30)
(112, 26)


In [40]:
muestra_estab = pd.merge(muestra_estab,emicron[['id','CIIU_3']], on = 'id')
muestra_calle = pd.merge(muestra_calle,emicron[['id','CIIU_3']], on = 'id')

In [41]:
print(muestra_estab.shape)

print(muestra_calle.shape)

(112, 31)
(112, 27)


In [43]:
muestra_estab.to_excel(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Resultados etapa 2\muestra_seleccionada_establecimientos v2.xlsx", index = False)
muestra_calle.to_excel(r"D:\DANE - Contrato\2024\Operativo Barrido\Data\Resultados etapa 2\muestra_seleccionada_moviles v2.xlsx", index = False)

In [4]:
emicron_nueva = pd.read_sas(os.path.join(data_path,"emicron_2023.sas7bdat"), format = "sas7bdat")

In [5]:
emicron_nueva.shape

(81018, 518)